In [1]:
#with mobile_sam model
import cv2
import numpy as np
import onnxruntime as ort
import torch
from mobile_sam import sam_model_registry
from mobile_sam.utils.transforms import ResizeLongestSide
import time

# ---------------- Load ONNX Encoder & Decoder ----------------
encoder = ort.InferenceSession("mobile_sam_encoder.onnx", providers=["CPUExecutionProvider"])
decoder = ort.InferenceSession("mobile_sam_decoder.onnx", providers=["CPUExecutionProvider"])

# ---------------- Load Prompt Encoder ----------------
sam = sam_model_registry["vit_t"](checkpoint="/home/logan78/Desktop/adobe/models/mobile_sam.pt")
sam.to("cpu")
prompt_encoder = sam.prompt_encoder
transform = ResizeLongestSide(1024)

# ---------------- Load Image ----------------
image_path = "/home/logan78/Desktop/adobe/sean-pollock-PhYq704ffdA-unsplash.jpg"
orig = cv2.imread(image_path)
H, W = orig.shape[:2]
rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

# --- Resize + Pad to 1024 ---
resized = transform.apply_image(rgb).astype(np.float32)
tensor = torch.as_tensor(resized).permute(2, 0, 1)[None]  # 1x3xHresxWres

# Pad to 1024×1024
padded = torch.zeros((1, 3, 1024, 1024), dtype=torch.float32)
padded[:, :, :tensor.shape[2], :tensor.shape[3]] = tensor
padded = padded / 255.0

print(f"🧠 Original: {H}x{W} | Resized: {tensor.shape[2]}x{tensor.shape[3]} | Padding: {(1024-tensor.shape[2])}px bottom")

# ---------------- Run Encoder ----------------
t0 = time.time()
image_embedding = encoder.run(None, {"image": padded.numpy()})[0]
t1 = time.time()
print(f"✅ Encoder runtime: {t1 - t0:.3f}s | Embedding shape: {image_embedding.shape}")

# ---------------- Mouse Click Interaction ----------------
clicks = []

def mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        clicks.clear()
        clicks.append((x, y))

cv2.namedWindow("MobileSAM Live")
cv2.setMouseCallback("MobileSAM Live", mouse)

# ---------------- Live Loop ----------------
while True:
    frame = orig.copy()

    if clicks:
        px, py = clicks[0]

        # Convert click → model coordinates
        pt = np.array([[px, py]])
        pt = torch.as_tensor(transform.apply_coords(pt, (H, W)), dtype=torch.float32)[None]  # 1x1x2
        labels = torch.ones(1, dtype=torch.int64)[None]  # 1x1 positive point

        # Prompt Encoder
        with torch.no_grad():
            sparse, dense = prompt_encoder(points=(pt, labels), boxes=None, masks=None)
        sparse = sparse.detach().cpu().numpy()
        dense = dense.detach().cpu().numpy()

        # ---------------- Run Decoder ----------------
        t2 = time.time()
        mask, iou = decoder.run(None, {
            "image_embedding": image_embedding,
            "sparse_prompt": sparse,
            "dense_prompt": dense
        })
        t3 = time.time()
        print(f"🧩 Decoder runtime: {t3 - t2:.3f}s")

        # ---------------- Correct Cropping ----------------
        mask = mask[0][0]  # (256,256)
        new_h, new_w = tensor.shape[2] // 4, tensor.shape[3] // 4
        pad_y = (1024 - tensor.shape[2]) // 4
        pad_x = (1024 - tensor.shape[3]) // 4
        print(f"Cropping mask: valid {new_h}x{new_w}, pad_y={pad_y}, pad_x={pad_x}")

        mask = mask[:new_h, :new_w]

        # Resize back to original
        mask = cv2.resize(mask, (W, H))
        mask_bin = (mask > 0.5).astype(np.uint8)

        # Overlay mask in green
        frame[mask_bin == 1] = (0, 255, 0)

    cv2.imshow("MobileSAM Live", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cv2.destroyAllWindows()


/home/logan78/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/logan78/.local/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/logan78/.local/lib/python3.10/site-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/logan78/.local/lib/python3.10/site-packages/mobile_sam/modeling/tiny_vit_sam.py:65

🧠 Original: 1280x1920 | Resized: 683x1024 | Padding: 341px bottom
✅ Encoder runtime: 0.504s | Embedding shape: (1, 256, 64, 64)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/logan78/.local/lib/python3.10/site-packages/cv2/qt/plugins"


🧩 Decoder runtime: 0.073s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.099s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.075s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.080s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.081s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.072s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.074s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.139s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.100s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.101s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.088s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.086s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.094s
Cropping mask: valid 170x256, pad_y=85, pad_x=0
🧩 Decoder runtime: 0.095s
Cropping mas

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
import torch
from segment_anything import sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
import time

# -----------------------------------------------------------
# ⚙️ Load ONNX Encoder & Decoder
# -----------------------------------------------------------
encoder_path = "/home/logan78/Desktop/adobe/onnx_fp16/sam_vit_b_encoder_fp16.onnx"
decoder_path = "/home/logan78/Desktop/adobe/onnx_fp16/sam_vit_b_decoder_fp16.onnx"

encoder = ort.InferenceSession(encoder_path, providers=["CPUExecutionProvider"])
decoder = ort.InferenceSession(decoder_path, providers=["CPUExecutionProvider"])

print("✅ ONNX models loaded successfully!")

# -----------------------------------------------------------
# ⚙️ Load Prompt Encoder (PyTorch)
# -----------------------------------------------------------
checkpoint = "/home/logan78/Desktop/adobe/models/sam_vit_b_01ec64.pth"
sam = sam_model_registry["vit_b"](checkpoint=checkpoint)
sam.to("cpu").eval()
prompt_encoder = sam.prompt_encoder
transform = ResizeLongestSide(1024)
print("✅ Prompt encoder loaded.")

# -----------------------------------------------------------
# 🖼️ Load Image
# -----------------------------------------------------------
image_path = "/home/logan78/Desktop/adobe/sean-pollock-PhYq704ffdA-unsplash.jpg"
orig = cv2.imread(image_path)
H, W = orig.shape[:2]
rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

# Resize and pad to 1024×1024
resized = transform.apply_image(rgb).astype(np.float32)
tensor = torch.as_tensor(resized).permute(2, 0, 1)[None]
padded = torch.zeros((1, 3, 1024, 1024), dtype=torch.float32)
padded[:, :, :tensor.shape[2], :tensor.shape[3]] = tensor
padded /= 255.0

print(f"🧠 Image Info — Original: {H}x{W}, Resized: {tensor.shape[2]}x{tensor.shape[3]}, Pad bottom: {1024 - tensor.shape[2]}")

# -----------------------------------------------------------
# 🚀 Run Encoder
# -----------------------------------------------------------
t0 = time.time()
image_embedding = encoder.run(None, {"image": padded.numpy().astype(np.float16)})[0]
t1 = time.time()
print(f"✅ Encoder runtime: {t1 - t0:.3f}s | Embedding shape: {image_embedding.shape}")

# -----------------------------------------------------------
# 🖱️ Mouse Click Setup
# -----------------------------------------------------------
clicks = []

def mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        clicks.clear()
        clicks.append((x, y))
        print(f"🖱️ Clicked at: ({x}, {y})")

cv2.namedWindow("SAM ViT-B Live")
cv2.setMouseCallback("SAM ViT-B Live", mouse)

# -----------------------------------------------------------
# 🧠 Auto dtype function (for safety)
# -----------------------------------------------------------
def auto_cast(name, arr):
    """Ensure correct dtype based on ONNX input expectation."""
    for inp in decoder.get_inputs():
        if inp.name == name:
            if inp.type == "tensor(float16)":
                return arr.astype(np.float16)
            else:
                return arr.astype(np.float32)
    return arr

# -----------------------------------------------------------
# 🎯 Main Interactive Loop
# -----------------------------------------------------------
while True:
    frame = orig.copy()

    if clicks:
        px, py = clicks[0]

        # Convert click → SAM input coords
        pt = np.array([[px, py]])
        pt = torch.as_tensor(transform.apply_coords(pt, (H, W)), dtype=torch.float32)[None]
        labels = torch.ones(1, dtype=torch.int64)[None]

        # Prompt encoder → sparse + dense
        with torch.no_grad():
            sparse, dense = prompt_encoder(points=(pt, labels), boxes=None, masks=None)
        sparse = sparse.detach().cpu().numpy()
        dense = dense.detach().cpu().numpy()

        # ---------------------------------------------------
        # Run Decoder (with dtype auto fix)
        # ---------------------------------------------------
        t2 = time.time()
        mask, iou = decoder.run(None, {
            "image_embedding": auto_cast("image_embedding", image_embedding),
            "sparse_prompt": auto_cast("sparse_prompt", sparse),
            "dense_prompt": auto_cast("dense_prompt", dense)
        })
        t3 = time.time()
        print(f"🧩 Decoder runtime: {t3 - t2:.3f}s | IOU: {iou[0][0]:.3f}")

        # ---------------------------------------------------
        # Postprocess mask
        # ---------------------------------------------------
        mask = mask[0][0]  # (256,256)
        new_h, new_w = tensor.shape[2] // 4, tensor.shape[3] // 4
        mask = mask[:new_h, :new_w]
        mask = cv2.resize(mask, (W, H))
        mask_bin = (mask > 0.5).astype(np.uint8)

        # Green overlay
        frame[mask_bin == 1] = (0, 255, 0)

    cv2.imshow("SAM ViT-B Live", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

cv2.destroyAllWindows()


✅ ONNX models loaded successfully!
✅ Prompt encoder loaded.
🧠 Image Info — Original: 1280x1920, Resized: 683x1024, Pad bottom: 341
✅ Encoder runtime: 4.343s | Embedding shape: (1, 256, 64, 64)
🖱️ Clicked at: (445, 576)
🧩 Decoder runtime: 0.062s | IOU: 0.646


error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4086: error: (-215:Assertion failed) func != 0 in function 'resize'


: 